from: https://stackabuse.com/text-classification-with-bert-tokenizer-and-tf-2-0-in-python/

Some of the stuff in the article refer to running on Colab, but I will run everything local...

In [152]:
from tensorflow.keras import layers
import tensorflow_hub as hub
import tensorflow as tf

import pandas as pd
import numpy as np
import bert

import re
import random
import math

In [8]:
movie_reviews = pd.read_csv("/Users/druss/Downloads/IMDB Dataset.csv")
print(movie_reviews.isnull().values.any())
movie_reviews.shape

False


(50000, 2)

In [17]:
movie_reviews.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


define functions for text preprocessing....

In [45]:

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    
    return sentence

perform the preprocessing...

In [46]:
reviews = []
sentences = list(movie_reviews['review'])
for sen in sentences:
    reviews.append(preprocess_text(sen))


In [50]:
print(len(reviews))
type(reviews)

50000


list

ok, we have the text, now let's deal with the outcomes(labels)
they are either positive or negative...

In [55]:
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, movie_reviews['sentiment'])))
print(len(y))
print(y[0:10])

50000
[1 1 1 0 1 1 1 0 0 1]


In [57]:
print(reviews[10])
print(y[10])

Phil the Alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines At first it was very odd and pretty funny but as the movie progressed didn find the jokes or oddness funny anymore Its low budget film thats never problem in itself there were some pretty interesting characters but eventually just lost interest imagine this film would appeal to stoner who is currently partaking For something similar but better try Brother from another planet 
0


Finally, I can start using trying to use BERT....

In [64]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [74]:
print( tokenizer.tokenize("don't be so judgmental") )
print( tokenizer.convert_tokens_to_ids( tokenizer.tokenize("don't be so judgmental") ) )

# create a function for tokenizing reviews...
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

# tokenize all reviews
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

['don', "'", 't', 'be', 'so', 'judgment', '##al']
[2123, 1005, 1056, 2022, 2061, 8689, 2389]


In the prepare data for training section, the article discuss padding by batch...

I'm going to set the seed so my results are reproducible...

In [148]:
random.seed(4);
reviews_with_len = [[review, y[i], len(review)]
                 for i, review in enumerate(tokenized_reviews)]
random.shuffle(reviews_with_len)

# not sure why we sort AFTER shuffling.  
# maybe so that all reviews of length x are shuffled?
reviews_with_len.sort(key=lambda x: x[2])
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]

Not sure what this means but ...
convert the sorted dataset into a TensorFlow 2.0-compliant input dataset shape.

In [149]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))


In [153]:
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)


The previous 3 code cells should be run only once in order to preserve reproducability.  We are now ready to make our model!


The next cell creates a model.  I am not sure the this doesn't use a bert_layer...  but let's run with it.

In [154]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

set hyperparameters...

In [156]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5


In [162]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])
    

we have created an instance and compiled the model...  lets take a look...

In [164]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
1407/1407 [==============================] - 237s 168ms/step - loss: 0.3056 - accuracy: 0.8667
Epoch 2/5
1407/1407 [==============================] - 239s 170ms/step - loss: 0.1390 - accuracy: 0.9493
Epoch 3/5
1407/1407 [==============================] - 232s 165ms/step - loss: 0.0672 - accuracy: 0.9767
Epoch 4/5
1407/1407 [==============================] - 227s 161ms/step - loss: 0.0365 - accuracy: 0.9870
Epoch 5/5
1407/1407 [==============================] - 241s 171ms/step - loss: 0.0225 - accuracy: 0.9920


ok, this is just using the BERT tokenizer.  What I want is to use the BERT Embedings...  Let start with BERT take 3!